In [2]:
'''Train CIFAR10 with PyTorch.
Reference: https://github.com/kuangliu/pytorch-cifar/
'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from resnet import ResNet18
from utils import progress_bar
from regularizers import FourierRegularizer,NormalizedFourierRegularizer


###############################################

import argparse

parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--regularizer', default=None, type=str, choices = ['Normalized_PSD','LSF', 'MSF', 'HSF', 'ASF'], help='Fourier Regularizer')
parser.add_argument('--regularizer_lambda', default=0.5, type=float, help='regularizer weight')
parser.add_argument('--resume', '-r', action='store_true', help='resume from checkpoint')


args = parser.parse_args(args=["--regularizer", "Normalized_PSD"])# pass an empty list to simulate command-line arguments
################################################
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='/tmp', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='/tmp', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

# Model
print('==> Building model..')

net = ResNet18()
net = net.to(device)


# if device == 'cuda':

#     print("aasd 1")

#     net = torch.nn.DataParallel(net, device_ids=[0])
# #     net = torch.nn.parallel.DistributedDataParallel(net)
#     print("aasd 2")

#     cudnn.benchmark = True

if args.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)


if args.regularizer:
    if args.regularizer=='Normalized_PSD':
        args.regularizer = NormalizedFourierRegularizer(args.regularizer_lambda)
        args.regularizer.mode='Normalized_PSD'
    else:
        args.regularizer = FourierRegularizer(args.regularizer, args.regularizer_lambda) # init regularizer
#         args.regularizer.mode =''

# Training
    


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..
==> Initialized Normalized-Fourier-regularizer lambda: 0.50


In [3]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    train_reg_loss = 0
    regularizer_loss = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        if args.regularizer:
            inputs = inputs.requires_grad_(True)

        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
#         print("aaaaaa")
        outputs = net(inputs)
#         print("bbbbbb")
        
        loss = criterion(outputs, targets)

        if args.regularizer:
            regularizer_loss =  args.regularizer(inputs, loss)
            loss = loss + regularizer_loss
            
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_reg_loss += regularizer_loss
        
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Reg. Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), train_reg_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
        
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')

        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }

        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        
        if args.regularizer:
            torch.save(state, './checkpoint/ckpt_%s_%.2f.pth' % (args.regularizer.mode, args.regularizer_lambda))
        else:
            torch.save(state, './checkpoint/ckpt.pth')

        best_acc = acc


In [ ]:
for epoch in range(start_epoch, start_epoch+150):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0


C:\Users\lucky\anaconda3\envs\torch\lib\site-packages\torch\functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3491.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


 [================================================================>]  Step: 195ms | Tot: 1m31s | Loss: 5.107 | Reg. Loss: 2.716 | Acc: 17.798% (8899/5000 391/391 1  
 [================================================================>]  Step: 11ms | Tot: 1s221ms | Loss: 1.993 | Acc: 25.760% (2576/1000 100/100 
Saving..

Epoch: 1
 [================================================================>]  Step: 159ms | Tot: 1m31s | Loss: 4.326 | Reg. Loss: 2.366 | Acc: 25.948% (12974/5000 391/391   
 [================================================================>]  Step: 10ms | Tot: 1s241ms | Loss: 1.841 | Acc: 31.460% (3146/1000 100/100 
Saving..

Epoch: 2
 [================================================================>]  Step: 161ms | Tot: 1m31s | Loss: 4.197 | Reg. Loss: 2.351 | Acc: 31.502% (15751/5000 391/391 1  
 [================================================================>]  Step: 11ms | Tot: 1s125ms | Loss: 1.754 | Acc: 35.380% (3538/1000 100/100 
Saving..

Epoch: 3
 [=======